<a href="https://colab.research.google.com/github/E01x001/AoAI/blob/main/HealthCare_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from flask import Flask, request, jsonify

app = Flask(__name__)

# 부위별 스트레칭 방법 정의
stretching_info = {
    "neck": [
        "목을 천천히 좌우로 돌려주세요. (15초씩)",
        "머리를 오른쪽으로 기울여 귀가 어깨에 닿도록 하세요. (15초 후 반대쪽)",
        "턱을 가슴 쪽으로 당기며 목 뒤쪽을 늘려주세요. (15초 유지)"
    ],
    "shoulder": [
        "양쪽 어깨를 위아래로 천천히 움직이며 스트레칭하세요. (15초)",
        "팔을 앞뒤로 크게 돌려주세요. (15초씩)",
        "양팔을 등 뒤에서 맞잡고 가슴을 펴 주세요. (15초 유지)"
    ]
}

@app.route("/stretch", methods=["GET"])
def get_stretch():
    part = request.args.get("part", "neck")  # 기본값은 목 스트레칭
    stretches = stretching_info.get(part.lower(), ["해당 부위 스트레칭 정보가 없습니다."])

    return jsonify({"part": part, "stretches": stretches})

if __name__ == "__main__":
    app.run(port=5000)  # 로컬에서 실행


In [2]:
import os
from google.colab import userdata
os.environ["NGROK_AUTH_TOKEN"] = userdata.get('NGROK_AUTH_TOKEN')

In [3]:
# 사용자별 데이터 저장
user_data = {}

# 📌 1. 사용자 데이터 저장 (아픈 부위 & 알림 빈도)
@app.route('/set_user_data', methods=['POST'])
def set_user_data():
    data = request.json
    user_id = data.get("user_id")
    body_part = data.get("body_part")
    frequency = data.get("frequency")  # 알림 간격 (분 단위)

    if user_id not in user_data:
        user_data[user_id] = {}

    user_data[user_id]["body_part"] = body_part
    user_data[user_id]["frequency"] = frequency
    user_data[user_id]["last_check_date"] = str(datetime.today().date())  # 최근 상태 체크 날짜

    return jsonify({"message": "사용자 데이터 저장 완료!", "data": user_data[user_id]})

# 📌 2. 사용자 데이터 조회
@app.route('/get_user_data', methods=['GET'])
def get_user_data():
    user_id = request.args.get("user_id")
    return jsonify(user_data.get(user_id, "사용자 데이터 없음"))

# 📌 3. 3~5일 간격으로 상태 체크 (자동 메시지 전송)
def check_user_status():
    while True:
        today = datetime.today().date()

        for user_id, info in user_data.items():
            last_check = datetime.strptime(info["last_check_date"], "%Y-%m-%d").date()
            days_since_last_check = (today - last_check).days

            if days_since_last_check >= 3:  # 3일 이상 경과 시 메시지 전송
                print(f"✅ 사용자 {user_id}에게 상태 체크 메시지를 전송합니다!")
                message = f"{info['body_part']} 부위가 여전히 아픈가요?\n1️⃣ 여전히 아파요\n2️⃣ 괜찮아졌어요"
                send_kakao_message(user_id, message)
                user_data[user_id]["last_check_date"] = str(today)

        time.sleep(86400)  # 하루에 한 번 실행 (86400초)

In [ ]:
# # 📌 4. 카카오톡 메시지 전송 함수 (API 연동)
# def send_kakao_message(user_id, message):
#     kakao_token = "YOUR_KAKAO_ACCESS_TOKEN"  # 카카오 API 토큰 입력
#     url = "https://kapi.kakao.com/v2/api/talk/memo/default/send"
#     headers = {"Authorization": f"Bearer {kakao_token}"}
#     data = {"template_object": json.dumps({"object_type": "text", "text": message, "link": {}})}

#     response = requests.post(url, headers=headers, data=data)
#     print(f"카카오톡 메시지 전송 결과: {response.json()}")


In [4]:
# 📌 5. 상태 업데이트 (사용자가 '괜찮아졌어요' or '여전히 아파요' 선택)
@app.route('/update_status', methods=['POST'])
def update_status():
    data = request.json
    user_id = data.get("user_id")
    status = data.get("status")  # "ongoing" (여전히 아픔) or "resolved" (괜찮아짐)

    if user_id in user_data:
        user_data[user_id]["status"] = status

        if status == "resolved":
            # 스트레칭 부위 변경 (목 → 어깨 → 허리 순환)
            body_part_order = ["neck", "shoulder", "back", "legs"]
            current_part = user_data[user_id]["body_part"]
            new_part = body_part_order[(body_part_order.index(current_part) + 1) % len(body_part_order)]
            user_data[user_id]["body_part"] = new_part
            return jsonify({"message": "스트레칭 부위 업데이트 완료!", "new_body_part": new_part})
        else:
            return jsonify({"message": "기존 스트레칭 유지. 추가 스트레칭을 추천합니다!"})

    return jsonify({"error": "사용자 정보 없음"})

In [9]:
from pyngrok import ngrok
# 📌 6. ngrok을 사용하여 API 외부 접근 가능하게 만들기
def start_server_with_ngrok():
    ngrok.set_auth_token("NGROK_AUTH_TOKEN")  # ngrok 인증 토큰 입력
    public_url = ngrok.connect(5000).public_url
    print(f"ngrok 터널이 생성되었습니다: {public_url}")
    print(f"API 문서는 {public_url}/docs 에서 확인 가능")

    app.run(port=5000)


In [7]:
# 📌 7. 백그라운드에서 상태 체크 스케줄러 실행
thread = threading.Thread(target=check_user_status)
thread.start()

In [ ]:
# 📌 8. 서버 실행
start_server_with_ngrok()

ngrok 터널이 생성되었습니다: https://635c-34-75-14-140.ngrok-free.app
API 문서는 https://635c-34-75-14-140.ngrok-free.app/docs 에서 확인 가능
 * Serving Flask app 'HealthCare'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [14/Mar/2025 09:14:26] "POST / HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [14/Mar/2025 09:26:47] "POST / HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [14/Mar/2025 09:27:11] "GET / HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [14/Mar/2025 09:27:12] "GET /favicon.ico HTTP/1.1" 404 -
